In [ ]:
import os,zipfile
import tarfile,json

In [ ]:
zip_path = 'rrc_data/zips/'
zip_dest = 'rrc_data/images/'
json_path = 'rrc_data/json/'

In [ ]:
with zipfile.ZipFile(os.path.join(zip_path,'val.zip'), 'r') as file:
    file.extractall(zip_dest)
    
with zipfile.ZipFile(os.path.join(zip_path,'test_croppedv2.zip'), 'r') as file:
    file.extractall(zip_dest)
    
with tarfile.open(os.path.join(zip_path,'cropped_train.tar.gz')) as file:
    file.extractall(zip_dest)

In [ ]:
with open(os.path.join(json_path,'cropped_train_v1.json'))as f:
    data_train = json.load(f)
    
with open(os.path.join(json_path,'cropped_val_v1.json'))as f:
    data_val = json.load(f)

In [ ]:
train_path = 'data/train/'
val_path = 'data/val/'

In [ ]:
os.makedirs(val_path,exist_ok=True)
os.makedirs(train_path,exist_ok=True)

for entry in data_val:
    
    img_file = str(entry['text_id'])+'.jpg'
    
    if entry['transcription'].isascii() and os.path.getsize(os.path.join(zip_dest,'val',img_file)):
        
        os.symlink(os.path.join(zip_dest,'val',img_file),os.path.join(val_path,img_file))
        
        with open(os.path.join(val_path,img_file[:-3]+'txt'),'w')as f:
            f.write(entry['transcription'])
            


for entry in data_train:
    
    img_file = str(entry['text_id'])+'.jpg'
    
    if entry['transcription'].isascii() and os.path.getsize(os.path.join(zip_dest,'train',img_file)):
        
        os.symlink(os.path.join(zip_dest,'train',img_file),os.path.join(train_path,img_file))
        
        with open(os.path.join(train_path,img_file[:-3]+'txt'),'w')as f:
            f.write(entry['transcription'])

In [ ]:
os.remove(os.path.join(train_path,'4985023.jpg'))
os.remove(os.path.join(train_path,'4985023.txt'))